In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

## We have to load it everytime..Stupid..

In [1]:
# %load ../Library/LoggingForIPython.py
def printIpython(data,head=5,level=1):
    '''Level = 1,2,3,4 -> 4 is print everything'''
    currentLevel = 2
    if(level <= currentLevel ):
        print data.head(head),'\n'

# Logging Function

In [3]:
def myPrint(data,head=5,level=1):
    '''Level = 1,2,3,4 -> 4 is print everything'''
    currentLevel = 2
    if(level <= currentLevel ):
        print data.head(3),'\n'

## Test Log..Delete..

In [4]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

logging.error('hello!')
logging.debug('This is a debug message')
logging.info('this is an info message')
logging.warning('tbllalfhldfhd, warning.')

## Products..

In [5]:
#All columns can fit one line..
pd.set_option('expand_frame_repr', False)

products = pd.read_csv("../../input/producto_tabla.csv")
myPrint(products,3,2)
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=True)
myPrint(products,3,4)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
myPrint(products,3,4)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
myPrint(products,3,4)
w.head(3)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
myPrint(products,3,4)
products['short_name_processed'] = (products['short_name'].map(
    lambda x: " ".join([i for i in x.lower().split() if i not in nltk.corpus.stopwords.words("spanish")])))
myPrint(products,3,4)
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (
products['short_name_processed'].map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))
myPrint(products,3,4)
products["pieces"].fillna(1, inplace=True)
products.drop(0,inplace=True)
products.drop(["short_name","NombreProducto"],axis=1,inplace=True)
products.head(5)
#products.isnull().sum()
#products[pd.isnull(products).any(1)]
products[pd.isnull(products.brand)]
products[products.brand == 'BIM']
#Temporary solution..
products.fillna(products.mean(),inplace=True).head(5)
########################################
### Find missing values on internet!..
###Missing brand is bimbo!!!
######################################
#print products.shape
#print len(products.Producto_ID.unique())

   Producto_ID                          NombreProducto
0            0                       NO IDENTIFICADO 0
1            9               Capuccino Moka 750g NES 9
2           41  Bimbollos Ext sAjonjoli 6p 480g BIM 41 



,Producto_ID,brand,weight,pieces,short_name_processed
1,9,NES,750.0,1.0,capuccin mok
2,41,BIM,480.0,6.0,bimboll ext sajonjoli
3,53,LON,170.0,1.0,burrit sincr
4,72,TR,45.0,4.0,div tir mini doradit
5,73,BIM,540.0,1.0,pan multigran linaz


In [6]:
#train = pd.read_csv('../../input/train_100.csv',dtype={'Semana': np.int8, 'Agencia_ID': np.int32,
#'Canal_ID': np.int8,'Ruta_SAK': np.int32,'Cliente_ID': np.int32,'Producto_ID': np.int32
#                                    })
#test = pd.read_csv('../../input/test_100.csv',dtype={'Semana': np.int8, 'Agencia_ID': np.int32,
#'Canal_ID': np.int8,'Ruta_SAK': np.int32,'Cliente_ID': np.int32,'Producto_ID': np.int32
#                               })
    
train = pd.read_csv('../../input/train.csv',usecols=['Semana', 'Agencia_ID', 
                                                     'Canal_ID', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil'])
                                                      
test = pd.read_csv('../../input/test.csv',usecols=['id', 'Semana', 'Agencia_ID', 'Canal_ID', 'Cliente_ID', 'Producto_ID']
                                                      )



testIds = test['id']
test.drop(['id'],axis = 1,inplace=True)
trainY = train['Demanda_uni_equil']
trainX = train[test.columns.values]
print trainX.head(1)
print test.head(1)
trainTest = trainX.append(test,ignore_index=True)
print trainTest.head(1)
print "shape: " ,trainTest.shape
mergedTrainedTestProduct = trainTest.merge(products,on="Producto_ID",how="left")
mergedTrainedTestProduct.head(5)

   Semana  Agencia_ID  Canal_ID  Cliente_ID  Producto_ID
0       3        1110         7       15766         1212
   Semana  Agencia_ID  Canal_ID  Cliente_ID  Producto_ID
0      11        4037         1     4639078        35305
   Semana  Agencia_ID  Canal_ID  Cliente_ID  Producto_ID
0       3        1110         7       15766         1212
shape:  (81179715, 5)


,Semana,Agencia_ID,Canal_ID,Cliente_ID,Producto_ID,brand,weight,pieces,short_name_processed
0,3,1110,7,15766,1212,BIM,120.0,2.0,rol canel
1,3,1110,7,15766,1216,BIM,135.0,2.0,rol glass
2,3,1110,7,15766,1238,BIM,140.0,2.0,panquecit got choc
3,3,1110,7,15766,1240,BIM,125.0,4.0,mantec vainill
4,3,1110,7,15766,1242,BIM,105.0,6.0,donit espolvor


In [8]:
print mergedTrainedTestProduct.dtypes
print train.dtypes

Semana                    int64
Agencia_ID                int64
Canal_ID                  int64
Cliente_ID                int64
Producto_ID               int64
brand                    object
weight                  float64
pieces                  float64
short_name_processed     object
dtype: object
Semana               int64
Agencia_ID           int64
Canal_ID             int64
Cliente_ID           int64
Producto_ID          int64
Demanda_uni_equil    int64
dtype: object


In [9]:
rowsize = test.shape[0]

del trainTest,test,trainX

gc.collect()

165

In [10]:
#brand names are single..
#print products["brand"][products["brand"].astype(str).map(lambda x:len(x.split()))>1]

In [11]:
#mergedTrainedTestProduct = mergedTrainedTestProduct[0:5]


#print mergedTrainedTestProduct

countvec = CountVectorizer()
shortname_sparse =  countvec.fit_transform(mergedTrainedTestProduct.short_name_processed)
#print shortname_sparse.toarray()
shortname_sparse = shortname_sparse.astype(np.int8)
shortname_sparse

<81179715x542 sparse matrix of type '<type 'numpy.int8'>'
	with 145774286 stored elements in Compressed Sparse Row format>

# Brand Dummy Variable..

In [11]:
brand_sparse = countvec.fit_transform(mergedTrainedTestProduct.brand)
#print brand_sparse.toarray()
brand_sparse = brand_sparse.astype(np.int8)
brand_sparse

<81179715x33 sparse matrix of type '<type 'numpy.int8'>'
	with 81179715 stored elements in Compressed Sparse Row format>

# Dropping unnecessary columns..

In [12]:
print mergedTrainedTestProduct.columns
mergedTrainedTestProduct.drop(mergedTrainedTestProduct.columns[[0,1,2,3,4,5,8]],axis=1,inplace=True)
mergedTrainedTestProduct = mergedTrainedTestProduct.astype(np.float16)
print mergedTrainedTestProduct.info()
mergedTrainedTestProduct.head(1)

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Cliente_ID', u'Producto_ID',
       u'brand', u'weight', u'pieces', u'short_name_processed'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 81179715 entries, 0 to 81179714
Data columns (total 2 columns):
weight    float16
pieces    float16
dtypes: float16(2)
memory usage: 929.0 MB
None


,weight,pieces
0,120.0,2.0


# Concatenating traintest data and sparse matrixes as sparse..

In [13]:
mergedTrainedTestProduct_sparse = coo_matrix(mergedTrainedTestProduct)

del mergedTrainedTestProduct
gc.collect()

#mergedTrainedTestProduct_sparse
#print fittedmergedTrainedTestProduct.astype(object).tocoo()
completeSparseData = hstack((mergedTrainedTestProduct_sparse, shortname_sparse,brand_sparse), format='csr')
#print completeSparseData.toarray()
completeSparseData
io.mmwrite("../../sparse/completeSparseData.mtx", completeSparseData)

In [14]:
del  shortname_sparse, brand_sparse
gc.collect()

0

# if indexing doesn't work after merging, use below but looks like slow..
## But, we shouldn't drop semana column, it's working like if semana > 9, it is test..

In [15]:
#Use if necessary!!..
#print completeSparseData.toarray()
#print (completeSparseData[:,0]>9).toarray()
#indices = np.where((completeSparseData[:,0]>9).toarray())[0]
#indices
#completeSparseData[indices,:]

In [16]:
#print completeSparseData.toarray(), "\n"
train_sparse = completeSparseData[0:train.shape[0]]
test_sparse = completeSparseData[train.shape[0]:]
#print train_sparse.get_shape()
test_sparse
io.mmwrite("../../sparse/test_sparse.mtx", test_sparse)

In [17]:
del completeSparseData
gc.collect()

0

## Root mean square logarithmic square evaluation function..

In [18]:
def rmsle_eval(y, y0):
    y0=y0.get_label()    
    assert len(y) == len(y0)
    return 'error',np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

# Prediction..

In [19]:
import xgboost as xgb

params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.1
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True

test_preds = np.zeros(rowsize)

xg_train = xgb.DMatrix(train_sparse, label=trainY)
xg_test = xgb.DMatrix(test_sparse)

del train_sparse,test_sparse,trainY
gc.collect()

watchlist = [(xg_train, 'train')]
num_rounds = 100


xgclassifier = xgb.train( params, xg_train, num_rounds, watchlist, feval = rmsle_eval, 
                         early_stopping_rounds= 20, verbose_eval = 10)
#fold_preds = np.around(xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration))
fold_preds = np.rint(xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration))

test_preds += fold_preds

submission = pd.DataFrame({'id':testIds, 'Demanda_uni_equil': test_preds})
submission[["id","Demanda_uni_equil"]].to_csv('../../submissions/' + 
                                              datetime.now().strftime('%Y-%m-%d-%H-%M-%S') +'.csv', index=False)

Will train until train error hasn't decreased in 20 rounds.
[0]	train-error:1.364675
[10]	train-error:0.916528
[20]	train-error:0.790882
[30]	train-error:0.765360
[40]	train-error:0.770126
[50]	train-error:0.783478
Stopping. Best iteration:
[32]	train-error:0.764680



In [20]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 2 columns):
Demanda_uni_equil    float64
id                   int64
dtypes: float64(1), int64(1)
memory usage: 106.8 MB
